<a href="https://colab.research.google.com/github/trista-paul/DS-Unit-2-Sprint-5-/blob/master/Trista's_Unit_2_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

!kaggle competitions download -c ds2-predictive-modeling-challenge

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/
  0% 0.00/3.81M [00:00<?, ?B/s]
100% 3.81M/3.81M [00:00<00:00, 63.0MB/s]
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 64.3MB/s]
  0% 0.00/948k [00:00<?, ?B/s]
100% 948k/948k [00:00<00:00, 133MB/s]
  0% 0.00/236k [00:00<?, ?B/s]
100% 236k/236k [00:00<00:00, 47.1MB/s]


# **Loading dataset, imports**

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels
import scipy
import datetime
import seaborn as sns
!pip install category_encoders
from category_encoders import OneHotEncoder
import sklearn
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.exceptions import DataConversionWarning
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
import warnings

    100% |████████████████████████████████| 61kB 2.3MB/s 


In [0]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip

Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
  inflating: test_features.csv       


In [0]:
test = pd.read_csv('test_features.csv')
train = pd.read_csv('train_features.csv')
y = pd.read_csv('train_labels.csv')
sample_submission = pd.read_csv('sample_submission.csv')

# **All Dataframe Shaping (no scroll)**

In [0]:
#First Model
train['date_recorded'] = pd.to_datetime(train['date_recorded'])
date_recorded = train[['date_recorded', 'id']]
train = train.drop(columns='date_recorded')
train = train.drop(columns=['wpt_name', 'subvillage', 'scheme_name'])
highcardinality = train[['id', 'funder', 'installer', 'ward']]
train = train.drop(columns=['funder', 'installer', 'ward'])
nans = train[['id', 'public_meeting', 'scheme_management', 'permit']]
train = train.drop(columns=['public_meeting', 'scheme_management', 'permit'])
highcardinality['lga'] = train['lga']
train = train.drop(columns='lga')
BaselineX = train.drop(columns='id')
Baseliney = y.drop(columns='id')
Baseliney = np.ravel(Baseliney)


test['date_recorded'] = pd.to_datetime(test['date_recorded'])
date_recorded_test = test[['date_recorded', 'id']]
test = test.drop(columns='date_recorded')
test = test.drop(columns=['wpt_name', 'subvillage', 'scheme_name'])
highcardinality_test = test[['id', 'funder', 'installer', 'ward', 'lga']]
test = test.drop(columns=['funder', 'installer', 'ward', 'lga'])
nans_test = test[['id', 'public_meeting', 'scheme_management', 'permit']]
test = test.drop(columns=['public_meeting', 'scheme_management', 'permit'])
Baselinetest = test.drop(columns='id')

In [0]:
#Second Model
funder_counts = highcardinality['funder'].value_counts()
frequent_funders = funder_counts[funder_counts > 1000].index
mask = highcardinality['funder'].isin(frequent_funders)
rows_with_frequent_funders = highcardinality[mask]
rows_without_frequent_funders = highcardinality[~mask]
for value in rows_without_frequent_funders['funder'].unique():
  rows_without_frequent_funders = rows_without_frequent_funders.replace({value:'Other'})
highcardinality = rows_with_frequent_funders.append(rows_without_frequent_funders)

highcardinality['installer'] = highcardinality['installer'].replace({'HESAWA':'Hesawa',
                               'Central Government':'Government', 'Central government': 'Government',
                               'DANID':'DANIDA', 'District council':'District Council',
                               'Gover':'Government', 'Commu':'Community', 'World vision': 'World Vision',
                               'Gove':'Government', 'DW':'DWE', 'Da':'DANIDA'})
installer_counts = highcardinality['installer'].value_counts()
frequent_installers = installer_counts[installer_counts > 1000].index
mask = highcardinality['installer'].isin(frequent_installers)
rows_with_frequent_installers = highcardinality[mask]
rows_without_frequent_installers = highcardinality[~mask]
for value in rows_without_frequent_installers['installer'].unique():
  rows_without_frequent_installers = rows_without_frequent_installers.replace({value:'Other'})
highcardinality = rows_with_frequent_installers.append(rows_without_frequent_installers)

highcardinality = highcardinality.drop(columns='ward')

lga_counts = highcardinality['lga'].value_counts()
frequent_lga = lga_counts[lga_counts > 1000].index
mask = highcardinality['lga'].isin(frequent_lga)
rows_with_frequent_lga = highcardinality[mask]
rows_without_frequent_lga = highcardinality[~mask]
for value in rows_without_frequent_lga['lga'].unique():
  rows_without_frequent_lga = rows_without_frequent_lga.replace({value:'Other'})
highcardinality = rows_with_frequent_lga.append(rows_without_frequent_lga)

nans = nans.replace({np.nan:0, True:1, False:0})
nans['scheme_management'] = nans['scheme_management'].replace({0:'None'})

date_recorded['year'] = date_recorded['date_recorded'].dt.year

In [0]:
train = train.merge(highcardinality, on='id', how='inner')
train = train.merge(nans, on='id', how='inner')
train = train.merge(date_recorded, on='id', how='inner')
train = train.drop(columns='date_recorded')

In [0]:
train['construction_year'] = train['construction_year'].replace({0:train['year'].min()})
train['gps_height'] = train['gps_height'].replace({0:train['gps_height'].mean()})
train['years_from_installation'] = train['year'] - train['construction_year']
train = train.drop(columns=['longitude', 'latitude', 'region', 'payment',
                              'extraction_type', 'extraction_type_group',
                              'quantity_group', 'source_type', 'waterpoint_type',
                              'recorded_by', 'id'])
pd.set_option('display.max_columns', None)
train.head()

,amount_tsh,gps_height,num_private,basin,region_code,district_code,population,construction_year,extraction_type_class,management,management_group,payment_type,water_quality,quality_group,quantity,source,source_class,waterpoint_type_group,funder,installer,lga,public_meeting,scheme_management,permit,year,years_from_installation
0,6000.0,1390.000000,0,Lake Nyasa,11,5,109,1999,gravity,vwc,user-group,annually,soft,good,enough,spring,groundwater,communal standpipe,Other,Other,Other,1,VWC,0,2011,12
1,0.0,1399.000000,0,Lake Victoria,20,2,280,2010,gravity,wug,user-group,never pay,soft,good,insufficient,rainwater harvesting,surface,communal standpipe,Other,Other,Other,0,Other,1,2013,3
2,25.0,686.000000,0,Pangani,21,4,250,2009,gravity,vwc,user-group,per bucket,soft,good,enough,dam,surface,communal standpipe,Other,Other,Other,1,VWC,1,2013,4
3,0.0,263.000000,0,Ruvuma / Southern Coast,90,63,58,1986,submersible,vwc,user-group,never pay,soft,good,dry,machine dbh,groundwater,communal standpipe,Other,Other,Other,1,VWC,1,2013,27
4,0.0,668.297239,0,Lake Victoria,18,1,0,2002,gravity,other,other,never pay,soft,good,seasonal,rainwater harvesting,surface,communal standpipe,Other,Other,Other,1,None,1,2011,9


In [0]:
#date_recorded_test, highcardinality_test, nans_test, test

funder_counts = highcardinality_test['funder'].value_counts()
frequent_funders = funder_counts[funder_counts > 1000].index
mask = highcardinality_test['funder'].isin(frequent_funders)
rows_with_frequent_funders = highcardinality_test[mask]
rows_without_frequent_funders = highcardinality_test[~mask]
for value in rows_without_frequent_funders['funder'].unique():
  rows_without_frequent_funders = rows_without_frequent_funders.replace({value:'Other'})
highcardinality_test = rows_with_frequent_funders.append(rows_without_frequent_funders)

highcardinality_test['installer'] = highcardinality_test['installer'].replace({'HESAWA':'Hesawa',
                               'Central Government':'Government', 'Central government': 'Government',
                               'DANID':'DANIDA', 'District council':'District Council',
                               'Gover':'Government', 'Commu':'Community', 'World vision': 'World Vision',
                               'Gove':'Government', 'DW':'DWE', 'Da':'DANIDA'})
installer_counts = highcardinality_test['installer'].value_counts()
frequent_installers = installer_counts[installer_counts > 1000].index
mask = highcardinality_test['installer'].isin(frequent_installers)
rows_with_frequent_installers = highcardinality_test[mask]
rows_without_frequent_installers = highcardinality_test[~mask]
for value in rows_without_frequent_installers['installer'].unique():
  rows_without_frequent_installers = rows_without_frequent_installers.replace({value:'Other'})
highcardinality_test = rows_with_frequent_installers.append(rows_without_frequent_installers)

highcardinality_test = highcardinality_test.drop(columns='ward')

lga_counts = highcardinality_test['lga'].value_counts()
frequent_lga = lga_counts[lga_counts > 1000].index
mask = highcardinality_test['lga'].isin(frequent_lga)
rows_with_frequent_lga = highcardinality_test[mask]
rows_without_frequent_lga = highcardinality_test[~mask]
for value in rows_without_frequent_lga['lga'].unique():
  rows_without_frequent_lga = rows_without_frequent_lga.replace({value:'Other'})
highcardinality_test = rows_with_frequent_lga.append(rows_without_frequent_lga)

nans_test = nans_test.replace({np.nan:0, True:1, False:0})
nans_test['scheme_management'] = nans_test['scheme_management'].replace({0:'None'})

date_recorded_test['year'] = date_recorded_test['date_recorded'].dt.year

test = test.merge(highcardinality_test, on='id', how='inner')
test = test.merge(nans_test, on='id', how='inner')
test = test.merge(date_recorded_test, on='id', how='inner')
test = test.drop(columns='date_recorded')

test['construction_year'] = test['construction_year'].replace({0:test['year'].min()})
test['gps_height'] = test['gps_height'].replace({0:test['gps_height'].mean()})
test['years_from_installation'] = test['year'] - test['construction_year']
test = test.drop(columns=['longitude', 'latitude', 'region', 'payment',
                              'extraction_type', 'extraction_type_group',
                              'quantity_group', 'source_type', 'waterpoint_type',
                              'recorded_by', 'id'])
pd.set_option('display.max_columns', None)
test.head()

,amount_tsh,gps_height,num_private,basin,region_code,district_code,population,construction_year,extraction_type_class,management,management_group,payment_type,water_quality,quality_group,quantity,source,source_class,waterpoint_type_group,funder,installer,lga,public_meeting,scheme_management,permit,year,years_from_installation
0,0.0,1996.0,0,Internal,21,3,321,2012,other,parastatal,parastatal,never pay,soft,good,seasonal,rainwater harvesting,surface,other,Other,Other,Other,1,Parastatal,1,2013,1
1,0.0,1569.0,0,Pangani,2,2,300,2000,gravity,vwc,user-group,never pay,soft,good,insufficient,spring,groundwater,communal standpipe,Government Of Tanzania,DWE,Other,1,VWC,1,2013,13
2,0.0,1567.0,0,Internal,13,2,500,2010,other,vwc,user-group,never pay,soft,good,insufficient,rainwater harvesting,surface,other,Other,Other,Other,1,VWC,0,2013,3
3,0.0,267.0,0,Ruvuma / Southern Coast,80,43,250,1987,other,vwc,user-group,unknown,soft,good,dry,shallow well,groundwater,other,Other,Other,Other,0,VWC,1,2013,26
4,500.0,1260.0,0,Ruvuma / Southern Coast,10,3,60,2000,gravity,water board,user-group,monthly,soft,good,enough,spring,groundwater,communal standpipe,Other,Other,Other,0,Water Board,1,2013,13


# **Shaping Xtrain and ytrain, first pass**

In [0]:
pd.set_option('display.max_columns', None)
Xtrain.head(1)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [0]:
Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [0]:
Xtrain.isnull().sum()

id                           0
amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_

In [0]:
#drop date_recorded for baseline
#later, convert date_recorded to year and month
Xtrain['date_recorded'] = pd.to_datetime(Xtrain['date_recorded'])
date_recorded = Xtrain[['date_recorded', 'id']]
Xtrain = Xtrain.drop(columns='date_recorded')

In [0]:
#evaluate which columns could run into encoding problems
Xtrain.select_dtypes(exclude=np.number).nunique()

funder                    1897
installer                 2145
wpt_name                 37400
basin                        9
subvillage               19287
region                      21
lga                        125
ward                      2092
public_meeting               2
recorded_by                  1
scheme_management           12
scheme_name               2696
permit                       2
extraction_type             18
extraction_type_group       13
extraction_type_class        7
management                  12
management_group             5
payment                      7
payment_type                 7
water_quality                8
quality_group                6
quantity                     5
quantity_group               5
source                      10
source_type                  7
source_class                 3
waterpoint_type              7
waterpoint_type_group        6
dtype: int64

In [0]:
#wpt_name, subvillage, scheme name(mostly uniques and nans) appear unsalvageable
Xtrain = Xtrain.drop(columns=['wpt_name', 'subvillage', 'scheme_name'])

In [0]:
#drop funder, installer, ward to analyze later
highcardinality = Xtrain[['id', 'funder', 'installer', 'ward']]
Xtrain = Xtrain.drop(columns=['funder', 'installer', 'ward'])

In [0]:
#drop public_meeting, scheme_management, and permit, the remaining cols with nans
#two of these are binary and it makes sense to drop na, but it would complicate
#splitting and recombining the columns at this stage
#I would consider longitude and latitude 0 to be nans too, but I'll keep for now
nans = Xtrain[['id', 'public_meeting', 'scheme_management', 'permit']]
Xtrain = Xtrain.drop(columns=['public_meeting', 'scheme_management', 'permit'])

In [0]:
#The Xtrain dataset to one-hot encode
Xtrain.isnull().sum()

id                       0
amount_tsh               0
gps_height               0
longitude                0
latitude                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
population               0
recorded_by              0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
dtype: int64

In [0]:
Xtrain.select_dtypes(exclude=np.number).nunique()

basin                      9
region                    21
lga                      125
recorded_by                1
extraction_type           18
extraction_type_group     13
extraction_type_class      7
management                12
management_group           5
payment                    7
payment_type               7
water_quality              8
quality_group              6
quantity                   5
quantity_group             5
source                    10
source_type                7
source_class               3
waterpoint_type            7
waterpoint_type_group      6
dtype: int64

In [0]:
#missed lga as problematic
highcardinality['lga'] = Xtrain['lga']
Xtrain = Xtrain.drop(columns='lga')

In [0]:
#id is needed later, but isn't predictive
BaselineX = Xtrain.drop(columns='id')

In [0]:
ytrain.head(1)

,id,status_group
0,69572,functional


In [0]:
ytrain['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [0]:
Baseliney = ytrain.drop(columns='id')

In [0]:
Baseliney.isnull().sum()

status_group    0
dtype: int64

In [0]:
Baseliney = np.ravel(Baseliney)
Baseliney.shape

(59400,)

# **Shaping Xtest**

In [0]:
test['date_recorded'] = pd.to_datetime(test['date_recorded'])
date_recorded_test = test[['date_recorded', 'id']]
test = test.drop(columns='date_recorded')

test = test.drop(columns=['wpt_name', 'subvillage', 'scheme_name'])
highcardinality_test = test[['id', 'funder', 'installer', 'ward', 'lga']]
test = test.drop(columns=['funder', 'installer', 'ward', 'lga'])

nans_test = test[['id', 'public_meeting', 'scheme_management', 'permit']]
test = test.drop(columns=['public_meeting', 'scheme_management', 'permit'])

In [0]:
test.isnull().sum()

id                       0
amount_tsh               0
gps_height               0
longitude                0
latitude                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
population               0
recorded_by              0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
dtype: int64

In [0]:
#verify Xtrain and test have same column order
Xtrain.isnull().sum()

id                       0
amount_tsh               0
gps_height               0
longitude                0
latitude                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
population               0
recorded_by              0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
dtype: int64

In [0]:
Baselinetest = test.drop(columns='id')

# **Majority Class Baseline - all 'functional' - 54.30% Accuracy**

In [0]:
ypred = ['functional'] * len(Baseliney)
accuracy_score(Baseliney, ypred)

0.543080808080808

# **First Model - Logistic with Standard Scaler - 73.52% Accuracy**

In [0]:
BaselineX.shape

(59400, 28)

In [0]:
Baseliney.shape

(59400,)

In [0]:
Baselinetest.shape

(14358, 28)

In [0]:
Xtrain, Xval, ytrain, yval = train_test_split(BaselineX, Baseliney)
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((44550, 28), (14850, 28), (44550,), (14850,))

In [0]:
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

pipeline = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=5000, multi_class='auto')
)

pipeline.fit(Xtrain, ytrain)

ypred = pipeline.predict(Xval)
accuracy_score(yval,ypred)

0.7337373737373737

In [0]:
print(Xval.shape)
Baselinetest.shape

(14850, 28)


(14358, 28)

In [0]:
print(pipeline.predict(Baselinetest).shape)
submission = sample_submission.copy()
submission['status_group'] = pipeline.predict(Baselinetest)
submission.head()

(14358,)


,id,status_group
0,50785,non functional
1,51630,functional
2,17168,non functional
3,45559,non functional
4,49871,functional


In [0]:
submission.to_csv('tristagpaul-submission.csv', index=False)

In [0]:
!ls -lh *.csv

-rw-r--r-- 1 root root 237K Mar 13 14:58 sample_submission.csv
---------- 1 root root 4.7M Mar 11 14:42 test_features.csv
---------- 1 root root  20M Mar 11 14:42 train_features.csv
---------- 1 root root 1.1M Mar 11 14:42 train_labels.csv
-rw-r--r-- 1 root root 256K Mar 13 15:30 tristagpaul-submission.csv


In [0]:
from google.colab import files
files.download('tristagpaul-submission.csv')

# **Shaping Model 2**

In [0]:
highcardinality.head(1)

,id,funder,installer,ward,lga
0,69572,Roman,Roman,Mundindi,Ludewa


In [0]:
highcardinality['funder'].value_counts() #convert funders with less than 1000 projects to Other

Government Of Tanzania            9084
Danida                            3114
Hesawa                            2202
Rwssp                             1374
World Bank                        1349
Kkkt                              1287
World Vision                      1246
Unicef                            1057
Tasaf                              877
District Council                   843
Dhv                                829
Private Individual                 826
Dwsp                               811
0                                  777
Norad                              765
Germany Republi                    610
Tcrs                               602
Ministry Of Water                  590
Water                              583
Dwe                                484
Netherlands                        470
Hifab                              450
Adb                                448
Lga                                442
Amref                              425
Fini Water               

In [0]:
funder_counts = highcardinality['funder'].value_counts()
frequent_funders = funder_counts[funder_counts > 1000].index
mask = highcardinality['funder'].isin(frequent_funders)
rows_with_frequent_funders = highcardinality[mask]
rows_without_frequent_funders = highcardinality[~mask]

for value in rows_without_frequent_funders['funder'].unique():
  rows_without_frequent_funders = rows_without_frequent_funders.replace({value:'Other'})

highcardinality = rows_with_frequent_funders.append(rows_without_frequent_funders)

In [0]:
highcardinality['installer'].value_counts() #convert installers with less than 1000 projects as Other
#combine HESAWA with Hesawa, Central Government with Government, DANID with DANIDA

DWE                          17402
Government                    1825
RWE                           1206
Commu                         1060
DANIDA                        1050
KKKT                           898
Hesawa                         840
0                              777
TCRS                           707
Central government             622
CES                            610
Community                      553
DANID                          552
District Council               551
HESAWA                         539
LGA                            408
World vision                   408
WEDECO                         397
TASAF                          396
District council               392
Gover                          383
AMREF                          329
TWESA                          316
WU                             301
Dmdd                           287
ACRA                           278
World Vision                   270
SEMA                           249
DW                  

In [0]:
highcardinality['installer'] = highcardinality['installer'].replace({'HESAWA':'Hesawa',
                               'Central Government':'Government', 'Central government': 'Government',
                               'DANID':'DANIDA', 'District council':'District Council',
                               'Gover':'Government', 'Commu':'Community', 'World vision': 'World Vision',
                               'Gove':'Government', 'DW':'DWE', 'Da':'DANIDA'})

In [0]:
installer_counts = highcardinality['installer'].value_counts()
frequent_installers = installer_counts[installer_counts > 800].index
mask = highcardinality['installer'].isin(frequent_installers)
rows_with_frequent_installers = highcardinality[mask]
rows_without_frequent_installers = highcardinality[~mask]#numbers are quite different now. I could make cutoff 800.

In [0]:
len(rows_without_frequent_installers)

30725

In [0]:
for value in rows_without_frequent_installers['installer'].unique():
  rows_without_frequent_installers = rows_without_frequent_installers.replace({value:'Other'})

In [0]:
rows_without_frequent_installers['installer'].value_counts()

Other    30725
Name: installer, dtype: int64

In [0]:
highcardinality = rows_with_frequent_installers.append(rows_without_frequent_installers)

In [0]:
highcardinality.shape

(59400, 5)

In [0]:
highcardinality['ward'].value_counts() #too dispersed to be useful

Igosi               307
Imalinyi            252
Siha Kati           232
Mdandu              231
Nduruma             217
Kitunda             203
Mishamo             203
Msindo              201
Chalinze            196
Maji ya Chai        190
Usuka               187
Ngarenanyuki        172
Chanika             171
Vikindu             162
Mtwango             153
Matola              145
Zinga/Ikerege       141
Maramba             139
Wanging'ombe        139
Itete               137
Magomeni            135
Kikatiti            134
Ifakara             134
Olkokola            133
Maposeni            130
Mvomero             129
Igongolo            129
Mlangali            125
Nkoma               122
Nkungulu            121
                   ... 
Kitete                1
Rasbura               1
Uchindile             1
Igogo                 1
Matarawe              1
Linda                 1
Korongoni             1
Thawi                 1
Ukata                 1
Nyamtinga             1
Kihangimahuka   

In [0]:
highcardinality = highcardinality.drop(columns='ward')

In [0]:
highcardinality['lga'].value_counts()

Njombe              2503
Arusha Rural        1252
Moshi Rural         1251
Bariadi             1177
Rungwe              1106
Kilosa              1094
Kasulu              1047
Mbozi               1034
Meru                1009
Bagamoyo             997
Singida Rural        995
Kilombero            959
Same                 877
Kibondo              874
Kyela                859
Kahama               836
Magu                 824
Kigoma Rural         824
Maswa                809
Karagwe              771
Mbinga               750
Iringa Rural         728
Serengeti            716
Lushoto              694
Namtumbo             694
Songea Rural         693
Mpanda               679
Mvomero              671
Ngara                669
Ulanga               665
                    ... 
Newala               231
Bahi                 224
Kisarawe             223
Temeke               215
Rorya                210
Tarime               209
Ngorongoro           201
Kiteto               193
Shinyanga Urban      191


In [0]:
#lga seems very useful (compared to long and lat by themselves), so the threshold will be lower (500).
lga_counts = highcardinality['lga'].value_counts()
frequent_lga = lga_counts[lga_counts > 500].index
mask = highcardinality['lga'].isin(frequent_lga)
rows_with_frequent_lga = highcardinality[mask]
rows_without_frequent_lga = highcardinality[~mask]

for value in rows_without_frequent_lga['lga'].unique():
  rows_without_frequent_lga = rows_without_frequent_lga.replace({value:'Other'})

highcardinality = rows_with_frequent_lga.append(rows_without_frequent_lga)

In [0]:
highcardinality.shape

(59400, 4)

In [0]:
highcardinality.head(10)

,id,funder,installer,lga
7,54551,Rwssp,DWE,Kahama
11,50409,Danida,DANIDA,Namtumbo
16,48451,Unicef,DWE,Njombe
17,58155,Unicef,DWE,Njombe
22,58500,Unicef,DWE,Njombe
25,22308,Government Of Tanzania,DWE,Moshi Rural
27,20145,Hesawa,DWE,Karagwe
28,19685,Government Of Tanzania,District Council,Rombo
32,57238,Government Of Tanzania,DWE,Moshi Rural
36,67359,Danida,Government,Kyela


In [0]:
highcardinality.isnull().sum()

id           0
funder       0
installer    0
lga          0
dtype: int64

In [0]:
nans.head(1)

,id,public_meeting,scheme_management,permit
0,69572,True,VWC,False


In [0]:
nans.isnull().sum()

id                      0
public_meeting       3334
scheme_management    3877
permit               3056
dtype: int64

In [0]:
nans['public_meeting'].value_counts() #both public meeting and permit are True/False.

True     51011
False     5055
Name: public_meeting, dtype: int64

In [0]:
nans = nans.replace({np.nan:0, True:1, False:0})

In [0]:
nans['public_meeting'].value_counts()

1    51011
0     8389
Name: public_meeting, dtype: int64

In [0]:
nans['permit'].value_counts()

1    38852
0    20548
Name: permit, dtype: int64

In [0]:
nans['scheme_management'].value_counts() #put NaNs in None

VWC                 36793
WUG                  5206
0                    3877
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [0]:
nans['scheme_management'] = nans['scheme_management'].replace({0:None})

In [0]:
nans['scheme_management'] = nans['scheme_management'].replace({None:'None'})

In [0]:
nans['scheme_management'].value_counts()

VWC                 36793
WUG                  5206
None                 3878
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
Name: scheme_management, dtype: int64

In [0]:
nans.isnull().sum()

id                   0
public_meeting       0
scheme_management    0
permit               0
dtype: int64

In [0]:
date_recorded.head(1)

,date_recorded,id
0,2011-03-14,69572


In [0]:
date_recorded['year'] = date_recorded['date_recorded'].dt.year

In [0]:
date_recorded.head(1)

,date_recorded,id,year
0,2011-03-14,69572,2011


In [0]:
date_recorded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 3 columns):
date_recorded    59400 non-null datetime64[ns]
id               59400 non-null int64
year             59400 non-null int64
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.4 MB


# **Second Model - Logistic with Standard Scaler - 73.75% Validation Accuracy**

In [0]:
Xtrain, Xval, ytrain, yval = train_test_split(train, Baseliney)
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((44550, 26), (14850, 26), (44550,), (14850,))

In [0]:
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

pipeline = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=5000, multi_class='auto')
)

pipeline.fit(Xtrain, ytrain)

ypred = pipeline.predict(Xval)
accuracy_score(yval,ypred)

0.7408754208754209

In [0]:
print(pipeline.predict(test).shape)
submission = sample_submission.copy()
submission['status_group'] = pipeline.predict(test)
submission.head()

(14358,)


,id,status_group
0,50785,non functional
1,51630,functional
2,17168,non functional
3,45559,non functional
4,49871,functional


In [0]:
submission.to_csv('tristagpaul-submission2.csv', index=False)
from google.colab import files
files.download('tristagpaul-submission2.csv')

# **Model 3 - Logistic - Choosing features from 2 - 67.17%**

In [0]:
model = pipeline.named_steps['logisticregression']
feature_names = pipeline.named_steps['onehotencoder'].transform(Xtrain).columns
coefficients = pd.Series(model.coef_[0], feature_names)
coefficients.sort_values(ascending=False)

quantity_enough                             0.208738
basin_Lake Nyasa                            0.203569
extraction_type_class_handpump              0.194238
payment_type_per bucket                     0.132916
waterpoint_type_group_cattle trough         0.109420
amount_tsh                                  0.104037
lga_Njombe                                  0.097971
scheme_management_Water Board               0.094598
lga_Kasulu                                  0.089682
management_private operator                 0.085534
construction_year                           0.080682
permit                                      0.078130
source_rainwater harvesting                 0.070422
installer_Hesawa                            0.064037
waterpoint_type_group_improved spring       0.062217
management_wug                              0.061370
basin_Pangani                               0.059814
waterpoint_type_group_communal standpipe    0.059196
public_meeting                              0.

In [0]:
coefficientsdf = pd.DataFrame(coefficients)
coefficientsdf = coefficientsdf.reset_index(drop=False)
coefficientsdf = coefficientsdf.rename(columns={0:'values'})
coefficientsdf.head()

,index,values
0,basin_Lake Tanganyika,-0.087372
1,basin_Internal,0.002628
2,basin_Lake Victoria,-0.033910
3,basin_Ruvuma / Southern Coast,-0.062156
4,basin_Pangani,0.059814


In [0]:
coefficientsdf[coefficientsdf['index'].str.contains(colname)]

,index,values
130,amount_tsh,0.104037


In [0]:
train.columns

Index(['amount_tsh', 'gps_height', 'num_private', 'basin', 'region_code',
       'district_code', 'population', 'construction_year',
       'extraction_type_class', 'management', 'management_group',
       'payment_type', 'water_quality', 'quality_group', 'quantity', 'source',
       'source_class', 'waterpoint_type_group', 'funder', 'installer', 'lga',
       'public_meeting', 'scheme_management', 'permit', 'year',
       'years_from_installation'],
      dtype='object')

In [0]:
reorder = ['amount_tsh', 'gps_height', 'num_private', 'basin', 'region_code',
           'district_code', 'population', 'construction_year', 'extraction_type_class',
           'management_group', 'management', 'payment_type', 'water_quality', 'quality_group', 'quantity',
           'source_class', 'source', 'waterpoint_type_group', 'funder', 'installer', 'lga',
           'public_meeting', 'scheme_management', 'permit', 'years_from_installation', 'year']

In [0]:
#average coefficient by category
coefs = []
name = []
for colname in reorder:
  coef = coefficientsdf[coefficientsdf['index'].str.contains(colname, regex=False)] #literal strings
  m = coef['values'].mean()
  coefs.append(m)
  name.append(colname)
pd.Series(coefs, name).sort_values(ascending=False)

amount_tsh                 0.104037
construction_year          0.080682
permit                     0.078130
public_meeting             0.058911
gps_height                 0.056588
population                 0.038419
district_code              0.033211
waterpoint_type_group      0.016052
source_class               0.006720
payment_type               0.006284
water_quality              0.004444
management_group           0.004106
quality_group              0.003413
source                     0.003218
management                 0.000625
installer                 -0.000234
lga                       -0.001605
scheme_management         -0.002286
basin                     -0.002838
funder                    -0.002855
extraction_type_class     -0.008731
num_private               -0.012486
quantity                  -0.017955
year                      -0.022829
region_code               -0.074662
years_from_installation   -0.085642
dtype: float64

In [0]:
#five best positive coors - amount_tsh, construction_year, permit, public_meeting, gps_height
#five best negative coors - years_from_installation, region_code, year, quantity, num_private
#that sounds wrong and like its selecting for non-categoricals

In [0]:
ThirdX = train[['amount_tsh', 'permit', 'public_meeting', 'gps_height',
                'years_from_installation', 'region_code', 'quantity', 'num_private']]
Thirdtest = test[['amount_tsh', 'permit', 'public_meeting', 'gps_height',
                'years_from_installation', 'region_code', 'quantity', 'num_private']]

In [0]:
Xtrain, Xval, ytrain, yval = train_test_split(ThirdX, Baseliney)
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((44550, 8), (14850, 8), (44550,), (14850,))

In [0]:
pipeline = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    StandardScaler(), 
    LogisticRegression(solver='lbfgs', max_iter=5000, multi_class='auto')
)

pipeline.fit(Xtrain, ytrain)

ypred = pipeline.predict(Xval)
accuracy_score(yval,ypred) #I thought so

0.6717845117845118

# **Second Model - Decision Tree - 70.18%**

In [0]:
pipeline = make_pipeline(
    OneHotEncoder(use_cat_names=True), 
    SimpleImputer(), 
    DecisionTreeClassifier(max_depth=3))

pipeline.fit(Xtrain, ytrain)
ypred = pipeline.predict(Xval)
accuracy_score(yval,ypred)

0.7017508417508418

In [0]:
ytrain

array(['non functional', 'functional', 'functional', ...,
       'non functional', 'functional', 'non functional'], dtype=object)

In [0]:
model = pipeline.named_steps['decisiontreeclassifier']
feature_names = pipeline.named_steps['onehotencoder'].transform(Xtrain).columns
class_names = ['functional', 'functional needs repair', 'non functional']

pd.Series(model.feature_importances_, feature_names).sort_values(ascending=False)

quantity_dry                             0.598164
waterpoint_type_group_other              0.305905
amount_tsh                               0.074503
source_spring                            0.016124
lga_Shinyanga Rural                      0.003217
funder_World Bank                        0.001384
scheme_management_WUG                    0.000703
years_from_installation                  0.000000
quality_group_unknown                    0.000000
quantity_seasonal                        0.000000
quantity_insufficient                    0.000000
quality_group_-1                         0.000000
quality_group_fluoride                   0.000000
quality_group_colored                    0.000000
quality_group_milky                      0.000000
quality_group_salty                      0.000000
quality_group_good                       0.000000
water_quality_-1                         0.000000
water_quality_fluoride abandoned         0.000000
water_quality_fluoride                   0.000000
